In [30]:
import pandas as pd
from glob import glob
import json, requests
import os

In [31]:
repo_root = '../../'

In [36]:
def downloadCDCCountyVaccinationData():
    raw = requests.get('https://covid.cdc.gov/covid-data-tracker/COVIDData/getAjaxData?id=vaccination_county_condensed_data')
    loadedJson = raw.json()['vaccination_county_condensed_data']
    vaccinationData = pd.DataFrame(loadedJson)
    outputDate = loadedJson[0]["Date"].replace('/','-')
    with open(os.path.join(repo_root, f'data-scripts/cdc/vaccination_county/vaccination_county_condensed_data_{outputDate}.json'), 'w') as outfile:
        json.dump(loadedJson, outfile)

    return glob(os.path.join(repo_root, 'data-scripts/cdc/vaccination_county/*.json'))

In [37]:
def parseVaccinationData(vaccinationDataList):
    for idx, file in enumerate(vaccinationDataList):
        with open(file) as f:
            data = json.load(f)
            if (type(data)==dict):
                data = data['vaccination_county_condensed_data']

        # Pull report date from first row of data
        currDate = data[0]['Date']
        # Try to load in JSON to dataframe, pass iteration if fail

        # on first iteration, define new data frames
        # After that, define temporary dataframe sna dmerge below
        if idx == 0:
            vaccineAdministered2 = pd.DataFrame(data)[['FIPS','Series_Complete_Yes']]
            vaccineAdministered2.columns = ['fips',currDate]

        else:
            dailyVaccineAdministered2 = pd.DataFrame(data)[['FIPS','Series_Complete_Yes']]
            dailyVaccineAdministered2.columns = ['fips',currDate]

            # Merge
            vaccineAdministered2 = vaccineAdministered2.merge(dailyVaccineAdministered2, on=["fips"])

    return vaccineAdministered2

In [38]:
currentFiles = downloadCDCVaccinationData()

In [41]:
parsedDf = parseVaccinationData(currentFiles)
parsedDf.to_csv(os.path.join(repo_root, 'public/csv/vaccine_fully_vaccinated_cdc.csv'),index=False)